## <font color=blue> Python Lab #09 </font>: Understanding Entropy Coding: Part I
### *Run-Length Coding and LZW compression* 


### Homework submission: School MOOC system
### <font color=red> Due date: Dec. 8 (Wednesday), 24:00 </font>
#### <font color=red> NOTE </font>: 
#1 ipynb file format should be <font color=red> LAB09_yourIDnumber.ipynb </font> ( ex:  LAB09_20185433224234.ipynb ).
#2 If you don't follow the instructions #1, there is <font color=red>10 % penalty</font>.

#3 Any copied submission won't be graded. 
> 1) i.e., 0 point for every same copy. <br>
> 2) Also, slight modification of one version will be regarded as copy. </font>
>```python
>if copy found:
>    for all copies:
>        score = 0
>```    

----

__NOTE__:<br>
- 1 point for problem 1, 2, 4, 5
- 3 points for problem 3, 6


### Run-Length Coding

Run-Length encoding result of a message `'DAAABBBBBBBBBBBCCCCD'` can be in several different formats, and we will use the `DA2B10C3D` format among them, which does not use any flag symbol. <br>
In practical implementation, the numbers in length are generally fixed length (or VLC) coded. In our implementation, however, with the above format, we should check the next code is a value or character. (e.g., 'D2' or 'DA'). Also, when we decode 'B10', since we use string data type, we also need to verify the next value is a number or character after '1'.
To avoid these complicated situation, therefore, it is better to use a `tuple` for simple en/decoding. In this case, the result will be `[('D', 0), ('A', 2), ('B', 10), ('C', 3), ('D', 0)]`. 


__Pseudocode__ for Run Length encoding:
`````````python
Loop: count = 0
      REPEAT
          get next symbol
          count = count + 1
      UNTIL (symbol unequal to next one)
      output symbol
      IF count > 1
          output count
      GOTO Loop
``````````

#### (1) Write a  RLC encoding function RlcEncode() according to the following conditions. And print out the RLC results of the given messages and compression ratio of each message.
#### Condition:
* Input arg: string
* Output (return value): a list containing tuples
* Tuple order: (symbol, run length)
* Input message: `'AAA'` and `'AAAAAAAAAAAAAAAAAAAAAABBBBBCCCCCCCCCAAACCCCCCD'`
* For compression ratio, we assume that 8 bits for each charater and each run. 

In [1]:
def RlcEncode(str):
    tup = []
    time = 0
    for i in range(len(str)):
        if i == 0 or (i>0 and str[i] == str[i-1]):
            time+=1
            if i == len(str)-1:
                tup.append((str[i],time))
        else:
            tup.append((str[i-1],time))
            time = 1
            if i == len(str)-1:
                tup.append((str[i],time))
    return tup

In [2]:
ex1 = 'AAA'
ex2 = 'AAAAAAAAAAAAAAAAAAAAAABBBBBCCCCCCCCCAAACCCCCCD'
print(RlcEncode(ex1))
print(RlcEncode(ex2))
rate1 = len(ex1)/(len(RlcEncode(ex1))*2)
rate2 = len(ex2)/(len(RlcEncode(ex2))*2)
print('compression ratio of message1 is : '+("%.2f" % rate1))
print('compression ratio of message2 is : '+("%.2f" % rate2))

[('A', 3)]
[('A', 22), ('B', 5), ('C', 9), ('A', 3), ('C', 6), ('D', 1)]
compression ratio of message1 is : 1.50
compression ratio of message2 is : 3.83


#### (2) Write a RLC decoding function RlcDecode() according to the following conditions. And print both input and output messages.
#### Condition:
* Input arg: A list containing tuples (output codes of both messages in problem (1))
* Output (return value): Symbols in string data type

In [3]:
def RlcDecode(lst):
    str = ''
    for i in range(len(lst)):
        for j in range(lst[i][1]):
            str+=lst[i][0]
    return str


In [25]:
str = input('input: ')
print('output: '+RlcDecode(RlcEncode(str)))
print()
str = input('input: ')
print('output: '+RlcDecode(RlcEncode(str)))

input: AAA
output: AAA

input: AAAAAAAAAAAAAAAAAAAAAABBBBBCCCCCCCCCAAACCCCCCD
output: AAAAAAAAAAAAAAAAAAAAAABBBBBCCCCCCCCCAAACCCCCCD


#### (3) Show 'BW_text.png' image and apply run length coding with the following conditions. And print 'original file size', 'RLC code length', and 'compression ratio'.

__Conditions__:
* Apply RLC for each line of the B&W image (the image is binary image, 0 or 255).
* Calculate the original file size of the image and print it in `bits`.
* Calculate the result of RLC also in `bits` and print it. 
* When you calculate RLC code length, we assume that __10 bits__ are assigned for each __run__, and __1 bit__ for each __character__ (i.e., B or W or in a tuple (B, 25)).
* Compression ratio of RLC = org file size / RLC bits

In [55]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

path=r"./BW_text.png"
im = Image.open(path).convert('L')
im1=np.array(im).tolist()
im2 = np.array(im).ravel()
code_len = 0
total =[]
for i in range(len(im1)):
    total.append(RlcEncode(im1[i]))
for j in range(len(total)):
    code_len+=len(total[j])


print('Original file size :'+"%.0f" % len(im2)+' bits')
print('RLC code length    : '+"%.0f" % (code_len*11)+' bits')
print('Compression ratio  :'+"%.2f" % (len(im2)/(code_len*11)))


Original file size :211968 bits
RLC code length    : 123101 bits
Compression ratio  :1.72


### LZW compression: Encoding
#### Pseudocode

``` python
BEGIN
	s = current input character;
	while not EOF
	{
		c = next input character;
		if s + c exists in the dictionary
			s = s + c;
		else
		{
			output the code for s;
			add string s + c to the dictionary with a new code;
			s = c
		}
	}
	output the code for s;
END

```

* The assumption of the above pseudocode is that we already have a defaut dictionary. Therefore, we need to make one default dictionary before LZW encoding. 
* One simple way to create a LZW dictionary is using the dictionary data structure in Python.
* One more thing to consider is that the types of symbol. For simplicity, let's assume that symbols are all upper case alphabets (ASCII code from 65 to 90).
* Instead of `while` loop, `for` loop can be easier in Python.
* Also, instead of EOF, we can use length of the input message to check the end of message string.

In [26]:
dict1 = {chr(i): i for i in range(65, 65 + 4)}
print(dict1)

{'A': 65, 'B': 66, 'C': 67, 'D': 68}


#### (4) Write a LZW encoding function LzwEncode() according to the following conditions. And print the final (updated) dictionary, LZW code stream, compression ratio.
#### Condition:
* Input arg: Message (`string`), the number of symbols (i.e., how many symbols we use from A to ?. If "the number of symbols" is 4, then we use A, B, C, and D).
* Output arg (return values): LZW code (`int`) in a `list` (e.g., [1, 2, 4, 5, 1])
* Use `while` loop (same way as the above pseudocode).
* Input message: `'ABABBABCABABBA'` (same as the lecture slide example)
* Initial dictionary: {'A': 1, 'B': 2, 'C': 3, ... up to the specified number of symbols}
* When you calculate compression ratio, assign __8 bits__ for each __charater__ and __4 bits__ for each __LZW codeword__.

In [6]:
def LzwEncode(str):
    print('*final dictionary*')
    print('-------------------')
    print('Code','  ','Letter')
    print('-------------------')
    ini_dict = {'A':1,'B':2,'C':3}
    List = []
    
    a = 0#切片的前端
    num = 3
    P = str[a]
    while a<len(str)-1:
        C = str[a+1]
        if (P+C) in ini_dict.keys():
            P = P+C
            a = a+1
        else:
            ini_dict.update({(P+C):num+1})
            num+=1
            List.append(ini_dict[P])
            P = C
            a = a+1
    List.append(ini_dict[P])
    for i in range(len(list(ini_dict.items()))):
        print(list(ini_dict.items())[i][1],'  ',list(ini_dict.items())[i][0])
    return List,ini_dict

lst,ini = LzwEncode('ABABBABCABABBA')

print()
print('LZW code stream: ',lst)
print('Compression ratio: ',round((8*len('ABABBABCABABBA'))/(4*len(lst)),2))


*final dictionary*
-------------------
Code    Letter
-------------------
1    A
2    B
3    C
4    AB
5    BA
6    ABB
7    BAB
8    BC
9    CA
10    ABA
11    ABBA

LZW code stream:  [1, 2, 4, 5, 2, 3, 4, 6, 1]
Compression ratio:  3.11


### LZW compression: Decoding
#### Pseudocode
``` python
BEGIN
	s = NIL;
	while not EOF
	{
		k = next input code;             # k is a codeword, i.e., 1, 2, 3, etc.
		entry = dictionary entry for k;  # entry is symbol(s), i.e., A, B, AB, etc.
		output entry;
		if ( s != NIL) 		
			add string s + entry[0] to dictionary with a new code; # entry[0]: the 1st letter
		s = entry;
	}
END
```

#### Tip
* Assuming our dictionary is formed with (symbol, code) pairs like {'A': 1, 'B': 2, 'C': 3}, it is easy to get a value by a key. However, we need to get a key by a value. In this case, we need to think a bit. Let's see the below examples.

#### Getting a value by a key

In [39]:
myDict =  {'A': 1, 'B': 2, 'C': 3}
print(myDict)
print()
print(myDict['C'])

{'A': 1, 'B': 2, 'C': 3}

3


#### Getting a key by a value
Unfortunately, there is no built-in function to get a key by a value in dictionary.
Then, we can make one.

But do we really need this new function? <br>
**What if...?** <br>
There is a really easy way to solve this problem **without** any new function. <br>
If you fully understand the LZW decoding procedure, then you will know. <br>
Figure it out by yourself. &#128522;

#### (5) Write a LZW decoding function LzwDecode() according to the following conditions. And print the final (updated) dictionary, LZW code stream, the original message, and decoded message.
#### Condition:
* Input arg: LZW code stream (`list`), the number of symbols
* Output arg (return values): Symbols (`string`) 
* Use `while` loop (same way as the above pseudocode).
* Input code stream: The resulting code stream of problem (3)
* Initial dictionary: Same codeword mapping as the above LZW encoder

In [7]:
def LzwDecode(List):
    lst1 = list(ini.items())
    a = 0
    decode = ''
    while a<len(List):
        for i in range(len(lst1)):
            if List[a]==lst1[i][1]:
                decode+=lst1[i][0]
        a+=1
    return decode

lst,ini = LzwEncode('ABABBABCABABBA')
print()
print('Encoded stream:',lst)
print('Orginal message: ','ABABBABCABABBA')
print('Decoded message: ',LzwDecode(lst))

*final dictionary*
-------------------
Code    Letter
-------------------
1    A
2    B
3    C
4    AB
5    BA
6    ABB
7    BAB
8    BC
9    CA
10    ABA
11    ABBA

Encoded stream: [1, 2, 4, 5, 2, 3, 4, 6, 1]
Orginal message:  ABABBABCABABBA
Decoded message:  ABABBABCABABBA


#### (6) Slightly modify the LzwEncode() function and apply it to 'kf_panda.png' image with the following conditions. And print 'original file size', 'LZW code length', and 'compression ratio'.
__Conditions__:
* Since we are dealing with an image, the number of symbols is 256.
* In this problem, we assume that the size of dictionary is fixed to 1024, in other words, whenever the updated dictionary index equals to 1024, we need to initialize the dictionary to 256 length dictionary again (fixed 10 bits dictionary). In other words, 10 bits are assigned to each codeword of LZW code.


In [7]:
def LzwEncode1(str1):
    ini_dict = {str(i): i for i in range(256)}
    List = []
    P = ''
    a = 0
    while(a < len(str1)):
        if len(ini_dict) >= 1024:
            ini_dict = {str(i): i for i in range(256)}
        C = str(str1[a])
        if P + C in ini_dict:
            P += C
        else:
            List.append(ini_dict[P])
            ini_dict.update({ P+C : len(ini_dict)})
            P = C
        a += 1
    return List, ini_dict

In [8]:
import imageio
img1 = imageio.imread('kf_panda.png')
print('Original file size:', img1.size*8, 'bits')
lst1,ini1 = LzwEncode1(img1.ravel())
print('LZW code length   :',len(lst1)*10,'bits')
print('Compression ratio : ',round((img.size*8)/(len(lst1)*10),2))

Original file size: 512000 bits
LZW code length   : 335810 bits
Compression ratio :  1.52
